In [16]:
### REQUIRED USER INPUTS ####

#add to existing transit volumes?
keep_exist_vols = False

# mode definitions 
modes_GO_train = set('r')
modes_subway = set('ml')
modes_express_bus = set('gp')
modes_rapid_bus = set('qn')
modes_local = set('bsL')
modes_aux = set('yvwzut')

# demand Matrices 
demand_GO_train = "mf26"
demand_subway = "mf23"
demand_express_bus = "mf21"
demand_rapid_bus = "mf24"
demand_local = "mf22"
demand_ztsa = "mf25"

#  minimum time output 
min_time_mtx = "mf27"

# mode-specific times
GO_train_time = "mf60"
subway_time = "mf62"
express_bus_time = "mf64"
rapid_bus_time = "mf66"
local_time = "mf68"
ztsa_time = "mf69"

# scenario number
scen_num  = 107

# existing transit segment attributes for volumes
voltr_GO_train = "@voltr_gt"
voltr_subway = "@voltr_s"
voltr_express_bus = "@voltr_ex"
voltr_rapid_bus = "@voltr_rb"
voltr_local = "@voltr_l"
voltr_ztsa = "@voltr_z"

In [17]:
# HDR - Transit Assignment tool

# Performs multi-class assignment for the five transit classes.
# Extracts perceived and actual times.
# Determines the class with the minimum perceived time for each OD, 
# and uses that class's ACTUAL time for the minimum time matrix.

import inro.modeller as m
mm = m.Modeller()
eb = mm.emmebank
sc = eb.scenario(scen_num)
transit_assignment = mm.tool("inro.emme.transit_assignment.extended_transit_assignment")
matrix_results = mm.tool('inro.emme.transit_assignment.extended.matrix_results')
matrix_calculator = mm.tool('inro.emme.matrix_calculation.matrix_calculator')
network_results = mm.tool('inro.emme.transit_assignment.extended.network_results')

# create matrices and extra attributes
#  setup matrices
if not eb.matrix(min_time_mtx):
    eb.create_matrix(min_time_mtx)
eb.matrix(min_time_mtx).name = "Minimum.Transit.Time" 
    
#  setup extra attributes
if not sc.extra_attribute(voltr_GO_train):
    sc.create_extra_attribute('TRANSIT_SEGMENT',voltr_GO_train)
if not sc.extra_attribute(voltr_subway):
    sc.create_extra_attribute('TRANSIT_SEGMENT',voltr_subway)
if not sc.extra_attribute(voltr_express_bus):
    sc.create_extra_attribute('TRANSIT_SEGMENT',voltr_express_bus)
if not sc.extra_attribute(voltr_rapid_bus):
    sc.create_extra_attribute('TRANSIT_SEGMENT',voltr_rapid_bus)
if not sc.extra_attribute(voltr_local):
    sc.create_extra_attribute('TRANSIT_SEGMENT',voltr_local)
if not sc.extra_attribute(voltr_ztsa):
    sc.create_extra_attribute('TRANSIT_SEGMENT',voltr_ztsa)

assignment_classes = {
            "GO Train": {
                "demand_matrix": demand_GO_train,
                "time_matrix": GO_train_time,
                "required_mode": modes_GO_train,
                "other_modes": set(),
                "vol_att": voltr_GO_train
            },
            "Subway": {
                "demand_matrix": demand_subway,
                "time_matrix": subway_time,
                "required_mode": modes_subway,
                "other_modes": set(),
                "vol_att": voltr_subway
            },
            "Express Bus": {
                "demand_matrix": demand_express_bus,
                "time_matrix": express_bus_time,
                "required_mode": modes_express_bus,
                "other_modes": modes_rapid_bus | modes_local,
                "vol_att": voltr_express_bus
            },
            "Rapid Bus": {
                "demand_matrix": demand_rapid_bus,
                "time_matrix": rapid_bus_time,
                "required_mode": modes_rapid_bus,
                "other_modes":  modes_local,
                "vol_att": voltr_rapid_bus
            },
            "Local Bus": {
                "demand_matrix": demand_local,
                "time_matrix": local_time,
                "required_mode": modes_local,
                "other_modes": set(),
                "vol_att": voltr_local
            },
            "Zone To Station Access": {
                "demand_matrix": demand_ztsa,
                "time_matrix": ztsa_time,
                "required_mode": modes_local| modes_rapid_bus|modes_express_bus,
                "other_modes": set(),
                "vol_att": voltr_ztsa
            }
        }

def get_journey_levels(name,parameters):

    #Level 0
    Level0 = {
        "boarding_cost": None,
        "boarding_time": None,
        "waiting_time": None,
    }
    Level0['description'] = 'init'
    Level0['destinations_reachable'] = False 
    transition_rules = []
    for mode in parameters["required_mode"]:
        transition_rules.append({
        "mode": mode,
        "next_journey_level": 1
        })
    for mode in parameters["other_modes"]:
        transition_rules.append({
        "mode": mode,
        "next_journey_level": 0
        })
    Level0["transition_rules"] = transition_rules
    
    #Level1
    Level1 = {
        "boarding_cost": None,
        "boarding_time": None,
        "waiting_time": None,
    }
    Level1['description'] = name
    Level1['destinations_reachable'] = True 
    transition_rules = []

    for mode in parameters["required_mode"]|parameters["other_modes"]:
        transition_rules.append({
        "mode": mode,
        "next_journey_level": 1
        })
    Level1["transition_rules"] = transition_rules

    return[Level0,Level1]

for mode in assignment_classes:
    print "processing group " + mode
    journey_levels = get_journey_levels(mode,assignment_classes[mode])
    n_threads = 8
    spec = {
        "modes": list(assignment_classes[mode]["required_mode"]|assignment_classes[mode]["other_modes"]|modes_aux),
        "demand": assignment_classes[mode]["demand_matrix"],
        "waiting_time": {
            "headway_fraction": 1,
            "effective_headways": "@effective_hdw",
            "spread_factor": 1,
            "perception_factor": 2.5
        },
        "boarding_time": {
            "at_nodes": {
                "penalty": 0,
                "perception_factor":1
            },
            "on_lines": None
        },
        "boarding_cost": {
            "at_nodes": {
                "penalty": 0,
                "perception_factor": 1
            },
            "on_lines": None
        },
        "in_vehicle_time": {
            "perception_factor": "@invehicle_factor"
        },
        "in_vehicle_cost": None,
        "aux_transit_time": {
            "perception_factor": 2.5
        },
        "aux_transit_cost": None,
        "flow_distribution_at_origins": {
            "choices_at_origins": "OPTIMAL_STRATEGY",
            "fixed_proportions_on_connectors": None
        },
        "flow_distribution_at_regular_nodes_with_aux_transit_choices": {
            "choices_at_regular_nodes": "OPTIMAL_STRATEGY"
        },
        "flow_distribution_between_lines": {
            "consider_total_impedance": True
        },
        "connector_to_connector_path_prohibition": {
            "at_nodes": "ALL",
            "reassign_demand_to_alternate_path": False
        },
        "od_results": {
                "total_impedance": None
            },
        "journey_levels": journey_levels,
        "performance_settings": {
        "number_of_processors": n_threads
        },
        "type": "EXTENDED_TRANSIT_ASSIGNMENT"
    }
    transit_assignment(spec, scenario=sc, add_volumes=keep_exist_vols, save_strategies=True,class_name=mode)
       
    board_mtx = eb.available_matrix_identifier('FULL')
    eb.create_matrix(board_mtx)

    ivt_mtx = eb.available_matrix_identifier('FULL')
    eb.create_matrix(ivt_mtx)

    aux_mtx = eb.available_matrix_identifier('FULL')
    eb.create_matrix(aux_mtx)

    wait_mtx = eb.available_matrix_identifier('FULL')
    eb.create_matrix(wait_mtx)

    total = assignment_classes[mode]["time_matrix"]
        
    if not eb.matrix(total):
        eb.create_matrix(total)
        
    eb.matrix(total).name = (mode + " time").replace(" ", ".")

    spec = {
        "by_mode_subset": {
            "modes": list(assignment_classes[mode]["required_mode"]|assignment_classes[mode]["other_modes"]|modes_aux),
            "actual_total_boarding_times": board_mtx,
            "actual_in_vehicle_times": ivt_mtx,
            "actual_aux_transit_times": aux_mtx,
            "perceived_first_boarding_times": None,
            "perceived_in_vehicle_times": None,
            "perceived_aux_transit_times": None
        },
        "type": "EXTENDED_TRANSIT_MATRIX_RESULTS",
        "actual_first_waiting_times": wait_mtx,
        "perceived_first_waiting_times": None
    }
    matrix_results(spec, sc, mode)
    
    #save transit volumes on segments
    spec = {
        "on_links": None,
        "on_segments": {
            "transit_volumes": assignment_classes[mode]['vol_att']
        },
        "aggregated_from_segments": {
            "segment_selection": {
                "transit_lines": "all",
                "links": "all"
            }
        },
        "analyzed_demand": assignment_classes[mode]["demand_matrix"],
        "constraint": None,
        "type": "EXTENDED_TRANSIT_NETWORK_RESULTS"
    }
    network_results(spec, sc, mode)
    
    spec = {
        "type": "MATRIX_CALCULATION",
        "result": total,
        "expression": board_mtx + "+" + ivt_mtx + "+" + aux_mtx + "+" + wait_mtx,
    }
    matrix_calculator(spec)
       
    eb.delete_matrix(board_mtx)
    eb.delete_matrix(ivt_mtx)
    eb.delete_matrix(aux_mtx)
    eb.delete_matrix(wait_mtx)

    assignment_classes[mode]['total'] = total

#calculate minimum travel time matrix
spec = {
    "type": "MATRIX_CALCULATION",
    "result": min_time_mtx,
    "expression": "((" + GO_train_time + ".min." + subway_time + ")+"\
    + ztsa_time + ").min." + express_bus_time + ".min." + rapid_bus_time + ".min." + local_time
}
matrix_calculator(spec)

print "Completed Transit Assignment and Time Extraction."

processing group GO Train
processing group Rapid Bus
processing group Express Bus
processing group Zone To Station Access
processing group Local Bus
processing group Subway
Completed Transit Assignment and Time Extraction.


In [10]:
#Delete class times (optional)
for mode in assignment_classes:
    eb.delete_matrix(assignment_classes[mode]['actual'])
    eb.delete_matrix(assignment_classes[mode]['perceived'])
    
print "Deleted matrices"

KeyError: 'actual'